In [23]:
import petab
import pypesto
import amici
from amici.petab_objective import simulate_petab
from amici.petab_import import import_petab_problem
import pypesto.petab
import amici.petab_simulate
import numpy as np

file_path = "Petab_files/G4Modelling.yaml" #"/Users/lennart/Projects/Benchmark-Models-PEtab/Benchmark-Models/Boehm_JProteomeRes2014/Boehm_JProteomeRes2014.yaml" "expdata/Petab_files/G4Modelling.yaml"
petab_problem = petab.Problem.from_yaml(file_path)
simulator = amici.petab_simulate.PetabSimulator(petab_problem)
amici_model = import_petab_problem(petab_problem)
petab_problem.measurement_df = simulator.simulate(noise=True, noise_scaling_factor=1)
simulator.remove_working_dir()
importer = pypesto.petab.PetabImporter(petab_problem)
objective = importer.create_objective()
model = objective.amici_model
solver = objective.amici_solver
model.setSteadyStateSensitivityMode(amici.SteadyStateSensitivityMode.integrationOnly)
solver.setAbsoluteTolerance(1e-9)
solver.setRelativeTolerance(1e-9)
solver.setAbsoluteToleranceSteadyState(1e-9)
solver.setRelativeToleranceSteadyState(1e-9)



2022-07-25 21:42:42.310 - amici.petab_import - INFO - Successfully loaded model G4_conversionprocess from /Users/lennart/Projects/G4pypestoproblems/amici_models/G4_conversionprocess.
2022-07-25 21:42:42,310 - amici.petab_import - INFO - Successfully loaded model G4_conversionprocess from /Users/lennart/Projects/G4pypestoproblems/amici_models/G4_conversionprocess.
2022-07-25 21:42:42.313 - amici.petab_import - INFO - Compiling model amici_model68 to /var/folders/jk/5n_905l97fd7q66w0rrn_c6r0000gn/T/tmpv4fr3y6m.
2022-07-25 21:42:42,313 - amici.petab_import - INFO - Compiling model amici_model68 to /var/folders/jk/5n_905l97fd7q66w0rrn_c6r0000gn/T/tmpv4fr3y6m.
2022-07-25 21:42:42.316 - amici.petab_import - INFO - Importing model ...
2022-07-25 21:42:42,316 - amici.petab_import - INFO - Importing model ...
2022-07-25 21:42:42.317 - amici.petab_import - INFO - Model name is 'amici_model68'.
Writing model code to '/var/folders/jk/5n_905l97fd7q66w0rrn_c6r0000gn/T/tmpv4fr3y6m'.
2022-07-25 21:42:

running AmiciInstall
running build_ext
Changed extra_compile_args for unix to ['-std=c++14']
Building model extension in /private/var/folders/jk/5n_905l97fd7q66w0rrn_c6r0000gn/T/tmpv4fr3y6m
building 'amici_model68._amici_model68' extension
Testing SWIG executable swig4.0... FAILED.
Testing SWIG executable swig3.0... FAILED.
Testing SWIG executable swig... SUCCEEDED.
swigging swig/amici_model68.i to swig/amici_model68_wrap.cpp
swig -python -c++ -modern -outdir amici_model68 -I/Users/lennart/Projects/venvs/BA_env/lib/python3.9/site-packages/amici/swig -I/Users/lennart/Projects/venvs/BA_env/lib/python3.9/site-packages/amici/include -o swig/amici_model68_wrap.cpp swig/amici_model68.i
Deprecated command line option: -modern. This option is now always on.
creating build
creating build/temp.macosx-12-arm64-cpython-39
creating build/temp.macosx-12-arm64-cpython-39/swig
clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -isysroot /Libra

2022-07-25 21:42:56.830 - amici.petab_import - INFO - Successfully loaded model amici_model68 from /var/folders/jk/5n_905l97fd7q66w0rrn_c6r0000gn/T/tmpv4fr3y6m.
2022-07-25 21:42:56,830 - amici.petab_import - INFO - Successfully loaded model amici_model68 from /var/folders/jk/5n_905l97fd7q66w0rrn_c6r0000gn/T/tmpv4fr3y6m.
2022-07-25 21:42:56,952 - petab.lint - INFO - Checking SBML model...
2022-07-25 21:42:56,953 - petab.lint - INFO - Checking measurement table...
2022-07-25 21:42:56,956 - petab.lint - INFO - Checking condition table...
2022-07-25 21:42:56,956 - petab.lint - INFO - Checking observable table...
2022-07-25 21:42:56,958 - petab.lint - INFO - Checking parameter table...
2022-07-25 21:42:56,964 - petab.lint - INFO - PEtab format check completed successfully.
Using existing amici model in folder /Users/lennart/Projects/G4pypestoproblems/amici_models/G4_conversionprocess.
2022-07-25 21:42:56,967 - pypesto.petab.importer - INFO - Using existing amici model in folder /Users/lenna

In [27]:
pypesto_problem = importer.create_problem(objective)


history_options=pypesto.HistoryOptions(trace_record=True)
result = pypesto.optimize.minimize(pypesto_problem, filename=None, n_starts=10, history_options=history_options)

100%|██████████| 10/10 [00:26<00:00,  2.69s/it]


In [28]:
pypesto_problem.objective(np.asarray(petab_problem.x_nominal_scaled)[
          np.asarray(petab_problem.x_free_indices)]
    )

8334489.366081746

In [29]:
print("fvals from result")
print(result.optimize_result.get_for_key("fval"))
print("manually computed fvals:")
for i in range(10):
    print(pypesto_problem.objective(result.optimize_result.as_dataframe()["x"][i][
          np.asarray(petab_problem.x_free_indices)]))

2022-07-25 21:45:33,775 - py.warnings - WARNING - /Users/lennart/Projects/venvs/BA_env/lib/python3.9/site-packages/pypesto/result/optimize.py:335: UserWarning: get_for_key() is deprecated in favour of optimize_result['key'] and will be removed in future releases.
  warnings.warn(



fvals from result
[-155.6625335761543, 8.292609755644008, 22.847894460421394, 22.848003035742835, 23.086126041462038, 62.38328984065126, 103.9092376840906, 103.90923769263549, 103.90923769371086, 493.08221792274685]
manually computed fvals:
-155.6625335761543
8.292609755644008
22.847894460421394
22.848015889369115
23.08894277300316
62.38328995514529
103.9092376840906
103.90923769263549
103.90923769371086
493.08221792274685


In [30]:
result.optimize_result.as_dataframe()

,id,x,fval,grad,hess,res,sres,n_fval,n_grad,n_hess,n_res,n_sres,x0,fval0,history,exitflag,time,message,optimizer
0,4,"[-3.747081355885703, 6.673405397371247, -1.407...",-155.662534,"[173.1055552250764, 6.8283377877980485, -220.9...",None,None,None,317,317,0,0,0,"[-4.955127726314017, 7.570345821568713, -4.449...",3.552944e+15,<pypesto.objective.history.MemoryHistory objec...,2,10.431653,ABNORMAL_TERMINATION_IN_LNSRCH,<ScipyOptimizer method=L-BFGS-B options={'maxi...
1,7,"[-6.005150429725731, 7.555617891009735, -0.817...",8.292610,"[-2.2303155939400637e-05, -2.9238640265955196e...",None,None,None,100,100,0,0,0,"[-4.3238276595181695, 6.51795220812904, -2.422...",9.715707e+09,<pypesto.objective.history.MemoryHistory objec...,0,2.647460,CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH,<ScipyOptimizer method=L-BFGS-B options={'maxi...
2,0,"[1.3109578647082567, 3.7241863871990204, -2.29...",22.847894,"[0.0008973116012453014, -0.0008902935510785687...",None,None,None,87,87,0,0,0,"[1.3389911920009858, 3.72169553465811, -2.4133...",5.105320e+04,<pypesto.objective.history.MemoryHistory objec...,0,2.457675,CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH,<ScipyOptimizer method=L-BFGS-B options={'maxi...
3,2,"[-6.805922778932854, 2.531087457395974, -5.830...",22.848003,"[0.00242034576729111, -1.464829535398734e-07, ...",None,None,None,117,117,0,0,0,"[-9.66910592133333, 3.696729968288305, -5.6294...",2.875454e+16,<pypesto.objective.history.MemoryHistory objec...,0,3.651557,CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH,<ScipyOptimizer method=L-BFGS-B options={'maxi...
4,9,"[-3.9133661783748535, 5.722062988606967, -1.52...",23.086126,"[0.0085612511828419, -0.00015958931769838148, ...",None,None,None,164,164,0,0,0,"[-1.5089343512492732, 5.710673971663391, -3.28...",8.547703e+13,<pypesto.objective.history.MemoryHistory objec...,2,5.213001,ABNORMAL_TERMINATION_IN_LNSRCH,<ScipyOptimizer method=L-BFGS-B options={'maxi...
5,5,"[-5.776066651377437, 6.092718730830749, 2.1564...",62.383290,"[2.4700151661314637e-07, 5.605813248627104e-08...",None,None,None,53,53,0,0,0,"[-3.6056488950095833, 6.401105657957085, -0.01...",1.832657e+04,<pypesto.objective.history.MemoryHistory objec...,0,1.192892,CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH,<ScipyOptimizer method=L-BFGS-B options={'maxi...
6,8,"[-10.0, 10.0, 3.0, -10.0, 10.0, 3.0, -10.0, -9...",103.909238,"[1.365695277588027e-30, -9.214242186274992e-35...",None,None,None,25,25,0,0,0,"[-5.071875935854378, 4.13448990374577, 1.48558...",2.000101e+36,<pypesto.objective.history.MemoryHistory objec...,0,0.336634,CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL,<ScipyOptimizer method=L-BFGS-B options={'maxi...
7,1,"[3.0, 0.12298076762254315, -10.0, 2.9999966097...",103.909238,"[-9.721100631747837e-23, 2.0632209777068674e-2...",None,None,None,36,36,0,0,0,"[-4.015462578315612, 3.3763031862591952, 0.355...",2.868548e+09,<pypesto.objective.history.MemoryHistory objec...,0,0.311079,CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH,<ScipyOptimizer method=L-BFGS-B options={'maxi...
8,6,"[-10.0, 9.928215681456013, 3.0, 2.999999998382...",103.909238,"[2.291576390495515e-31, -2.054741282578478e-35...",None,None,None,33,33,0,0,0,"[1.6348063640117392, 6.596458789608718, -0.663...",2.400226e+10,<pypesto.objective.history.MemoryHistory objec...,0,0.453118,CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH,<ScipyOptimizer method=L-BFGS-B options={'maxi...
9,3,"[3.0, 0.0, -10.0, 3.0, 0.0, -10.0, -10.0, -10....",493.082218,"[-2.1486190581022503e-50, -1.0118942452546365e...",None,None,None,20,20,0,0,0,"[-3.4065460445846094, 6.0266361128922865, 0.26...",2.955632e+27,<pypesto.objective.history.MemoryHistory objec...,0,0.189867,CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH,<ScipyOptimizer method=L-BFGS-B options={'maxi...
